In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px
import matplotlib.pyplot as plt
from collections import namedtuple

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_degradated_20to40_component_only.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/SecondBatch/Analysis/rsv_25k.xlsx')
# df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/oligosdegradsample2-1.xlsx')
df.shape
df = thermo_df(df)
plot_zone(df)

In [ ]:
plt.figure(frameon=False)
_, ax = plt.subplots(figsize=(16, 9))
# ax.figure(figsize=(16, 12))
ax.set_axis_off()
df_main = df[df.Mass < 20000]
plt.scatter(df_main.Mass, df_main.RT)
plt.savefig('/Users/bryan/Downloads/df.png', transparent=True)

In [ ]:
bcr_5p = base_calling_random(df[df.Mass>22000])
plt, _ = plot_basecalling(*bcr_5p, annotate=True)

In [ ]:
bcr_5p = base_calling_random(df)
plt, _ = plot_basecalling(*bcr_5p, annotate=False)

# Single Filter

In [ ]:
mass_win_loc = 6500
mass_win_size = 100

In [ ]:
df_win = df[(df.Mass > mass_win_loc) & (df.Mass < mass_win_loc + mass_win_size)]
df_win.shape
plot_zones(df, df_win)

In [ ]:
def bin_range(df, bin_num):
    rt = df.RT
    if rt.empty:
        return 0.0, 0.0
    rt_min = rt.min()
    rt_max = rt.max()
    rt_height = rt_max - rt_min
#     print(rt_min, rt_max, rt_height)
    
    bin_size = rt_height / 5.
    bin_size = 0.5 if bin_size < 0.5 else bin_size

    bins = rt_height / rt_win_size
    
    rt_hi = rt_max - bin_size * (bin_num - 1)
    rt_lo = rt_hi - bin_size
#     print(type(rt_lo), type(bin_size))
    if np.less(rt_lo, rt_min):
        print('Reach the last bin.')
        rt_lo = rt_min
        rt_hi = rt_lo + bin_size
    print('df_win count {} RT lo-hi {}-{}'.format(df.shape[0], rt_lo, rt_hi))
    return rt_lo, rt_hi

In [ ]:
# rt_lo, rt_hi = bin_range(df_win, 1)
rt_lo, rt_hi = bin_range2(df_win, mass_win_loc, mass_win_loc+mass_win_size, f_hi2, f_lo2, 1)
df_filter = df_win[(df_win.RT<=rt_hi) & (df_win.RT>=rt_lo)]
print(df_filter.shape)
# plt = plot_zones(df_win, df_filter)
fig, ax = plt.subplots(1, figsize=(16, 12))
rect = create_rectangle(mass_win_loc, mass_win_loc+mass_win_size, rt_lo, rt_hi)
ax.add_patch(rect)
# ax.scatter(df_win.Mass, df_win.RT)
ax.scatter(df_sample.Mass, df_sample.RT)
ax.scatter(df_win.Mass, df_win.RT)

# Sliding Filter

In [ ]:
df.Mass.max()

In [ ]:
import matplotlib.patches as patches

In [ ]:
def create_rectangle(left, right, lo, hi):
    rect = patches.Rectangle((left, lo), right-left, hi-lo, color='r')
    return rect

In [ ]:
df_sample.RT.min(), df_sample.RT.max()

In [ ]:
filter_min = 0.3
filter_nums = 5

def bin_range2(df, left, right, f_hi, f_lo, bin_num):
    rt = df.RT
    if rt.empty:
        return 0.0, 0.0
#     rt_min = rt.min()
#     rt_max = rt.max()
    rt_min = f_lo(left)
    rt_min = 0.26 if rt_min < 0.26 else rt_min
    rt_max = f_hi(right)
    rt_max = 10.13 if rt_max > 10.13 else rt_max
    rt_height = rt_max - rt_min
#     print(rt_min, rt_max, rt_height)
    
    bin_size = rt_height / filter_nums
    bin_size = filter_min if bin_size < filter_min else bin_size

    bins = rt_height / rt_win_size
    
    rt_hi = rt_max - bin_size * (bin_num - 1)
    rt_lo = rt_hi - bin_size
#     print(type(rt_lo), type(bin_size))
    if np.less(rt_lo, rt_min):
        print('Reach the last bin.')
        rt_lo = rt_min
        rt_hi = rt_lo + bin_size
    print('df_win count {} winRT {}-{} filterRT lo-hi {}-{}'.format(df.shape[0], rt_min, rt_max, rt_lo, rt_hi))
    return rt_lo, rt_hi

In [ ]:
cursor = 0
mass_win_size = 10
bin_num = 2
last_rt_lo = 0.0

# plt.figure(figsize=(16, 9))
fig, ax = plt.subplots(1, figsize=(16, 9))
ax.scatter(df.Mass, df.RT)
filters = list()
while cursor < 20000:
    # do sth
#     print('processing win {}-{}'.format(cursor, cursor+mass_win_size))
    df_win = df[(df.Mass >= cursor) & (df.Mass < cursor + mass_win_size)]
    rt_lo, rt_hi = bin_range2(df_win, cursor, cursor+mass_win_size, f_hi2, f_lo2, bin_num)
    if rt_lo < last_rt_lo:
        rt_lo = last_rt_lo
        if rt_lo >= rt_hi:
            cursor += mass_win_size
            continue
            
    if np.isclose(rt_lo, 0.0) or np.isclose(rt_hi, 0.0):
        cursor += mass_win_size
        continue
    
    rect = create_rectangle(cursor, cursor+mass_win_size, rt_lo, rt_hi)
    ax.add_patch(rect)
    
    last_rt_lo = rt_lo
    df_filter = df_win[(df_win.RT>=rt_lo) & (df_win.RT<=rt_hi)]
#     print(cursor, df_filter.shape)
    filters.append(df_filter)
    cursor += mass_win_size

plt.show()

In [ ]:
df_filters = pd.concat(filters)
plt = plot_zones(df[df.Mass<20000], df_filters)
plt.plot(df_hi.Mass, f_hi2(df_hi.Mass))
plt.plot(df_lo.Mass, f_lo2(df_lo.Mass))
plt.savefig('/Users/bryan/Documents/Presentations/Jan 12 2021/filter_{}_bin_{}_filters_{}.png'.format(filter_min, bin_num, filter_nums), transparent=True)

In [ ]:
bcr_5p = base_calling_random(df_filters)
plt, _ = plot_basecalling(*bcr_5p, annotate=False)
plt.savefig('/Users/bryan/Documents/Presentations/Jan 12 2021/filter_bc_{}_bin_{}_filters_{}.png'.format(filter_min, bin_num, filter_nums), transparent=True)

In [ ]:
G = nx.DiGraph()

df_bcr = bcr_5p[0]
mass_pairs = bcr_5p[1]
for t in mass_pairs:
#     df_pair = df_bcr[df_bcr.Mass.isin(t)]
#     if df_pair.empty:
#         continue
#     plt.plot(df_pair.Mass, df_pair.RT, 'green')
#     print(t)
#     G.add_path([t[1], t[0]])
    G.add_edge(t[1], t[0], base=t[-1])

path = longest_path(G)
seqs = [G.edges[path[i], path[i+1]]['base'] for i in range(len(path)-1)]
print('longest sequence: ' + ''.join(seqs))
print(path)

In [ ]:
import networkx as nx
def longest_path(G):
    dist = {} # stores [node, distance] pair
    for node in nx.topological_sort(G):
        # pairs of dist,node for all incoming edges
        pairs = [(dist[v][0]+1,v) for v in G.pred[node]] 
        if pairs:
            dist[node] = max(pairs)
        else:
            dist[node] = (0, node)
    node,(length,_)  = max(dist.items(), key=lambda x:x[1])
    path = []
    while length > 0:
        path.append(node)
        length,node = dist[node]
    return list(reversed(path))

In [ ]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from descartes import PolygonPatch
import alphashape

def draw_fit_zone(df):
        df = df.sort_values(by='Mass')
        x = df['Mass']
        y = df['RT']
        plt.figure(figsize=(16, 12))
        plt.scatter(x, y)
        points = np.array([x, y]).T
        hull = ConvexHull(points)
#         fig, ax = plt.subplots(1, figsize=(16, 9))
        for simplex in hull.simplices:
            plt.plot(points[simplex, 0], points[simplex, 1], '--', color='green')

        z = np.polyfit(x, y, 2)
        def f(x):
            return z[0]*x*x + z[1]*x + z[2]
        y = f(x)
        plt.plot(x, y, "r-")

        plt.show()

#         return df_invalid

In [ ]:
draw_fit_zone(df[df.Mass < 20000])

In [ ]:
def alpha_shape_func(points, alpha, only_outer=True):
    """
    Compute the alpha shape (concave hull) of a set of points.
    :param points: np.array of shape (n,2) points.
    :param alpha: alpha value.
    :param only_outer: boolean value to specify if we keep only the outer border
    or also inner edges.
    :return: set of (i,j) pairs representing edges of the alpha-shape. (i,j) are
    the indices in the points array.
    """
    assert points.shape[0] > 3, "Need at least four points"

    def add_edge(edges, i, j):
        """
        Add a line between the i-th and j-th points,
        if not in the list already
        """
        if (i, j) in edges or (j, i) in edges:
            # already added
            assert (j, i) in edges, "Can't go twice over same directed edge right?"
            if only_outer:
                # if both neighboring triangles are in shape, it's not a boundary edge
                edges.remove((j, i))
            return
        edges.add((i, j))

    tri = Delaunay(points)
    edges = set()
    # Loop over triangles:
    # ia, ib, ic = indices of corner points of the triangle
    for ia, ib, ic in tri.vertices:
        pa = points[ia]
        pb = points[ib]
        pc = points[ic]
        # Computing radius of triangle circumcircle
        # www.mathalino.com/reviewer/derivation-of-formulas/derivation-of-formula-for-radius-of-circumcircle
        a = np.sqrt((pa[0] - pb[0]) ** 2 + (pa[1] - pb[1]) ** 2)
        b = np.sqrt((pb[0] - pc[0]) ** 2 + (pb[1] - pc[1]) ** 2)
        c = np.sqrt((pc[0] - pa[0]) ** 2 + (pc[1] - pa[1]) ** 2)
        s = (a + b + c) / 2.0
        area = np.sqrt(s * (s - a) * (s - b) * (s - c))
        circum_r = a * b * c / (4.0 * area)
        if circum_r < alpha:
            add_edge(edges, ia, ib)
            add_edge(edges, ib, ic)
            add_edge(edges, ic, ia)
    return edges

In [ ]:
from scipy.spatial import Delaunay
df_win['RT1'] = df_win.RT * 1000
points = np.array(df_win[['Mass', 'RT1']])
# print(points)
edges = alpha_shape_func(points, 0.25, only_outer=False)
print(edges)

In [ ]:
idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df[df.Mass < 20000]
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
df_chosen.shape

In [ ]:
df_hi = df_chosen
# df_lo = df_chosen

In [ ]:
x = df_hi.Mass
y = df_hi.RT
z = np.polyfit(x, y, 4)
def f_hi2(x):
    return z[0] * x**4 + z[1]*x**3 + z[2]*x**2 + z[3]*x + z[4] + 0.1
y = f_hi2(x)
plt.figure(figsize=(16, 12))
plt.scatter(df_sample.Mass, df_sample.RT)
plt.plot(x, y, "r-")

plt.scatter(x, y)
plt.show()

In [ ]:
x = df_lo.Mass
y = df_lo.RT
# z_lo2 = np.polyfit(x, y, 4)
z_lo2 = np.polyfit(x, y, 4)
def f_lo2(x):
    return z_lo2[0] * x**4 + z_lo2[1]*x**3 + z_lo2[2]*x**2 + z_lo2[3]*x + z_lo2[4] - 0.1
#     return z_lo2[0] * x**4 + z_z_lo2lo[1]*x**3 + z_lo2[2]*x**2 + z_lo2[3]*x + z_lo2[4] - 0.1
#     return z_lo[0] * x**3 + z_lo[1]*x**2 + z_lo[2]*x + z_lo[3]
y = f_lo2(x)
plt.figure(figsize=(16, 12))
plt.scatter(df_sample.Mass, df_sample.RT)
plt.plot(x, y, "r-")
# plt.scatter(x, y)
plt.show()

In [ ]:
# plt = plot_zones(df_chosen, df_lo)
_, ax = plt.subplots(figsize=(16, 12))
ax.scatter(df_sample.Mass, df_sample.RT)

# ax.scatter(df_hi.Mass, df_hi.RT, c='red')
ax.plot(df_hi.Mass, f_hi2(df_hi.Mass), c='red')

# ax.scatter(df_lo.Mass, df_lo.RT, c='green')
ax.plot(df_lo.Mass, f_lo2(df_lo.Mass), c='green')

# Bezier Curve
# nodes_hi = np.array(df_hi[['Mass', 'RT']]).T
# num_hi = df_chosen.shape[0]
# curve_hi = bezier.Curve(nodes_hi, degree=num_hi-1)
# curve_hi.plot(num_hi, ax=ax)

# nodes_lo = np.array(df_lo[['Mass', 'RT']]).T
# num_lo = df_lo.shape[0]
# curve_lo = bezier.Curve(nodes_lo, degree=num_lo-1)
# curve_lo.plot(num_lo, ax=ax)

In [ ]:
import bezier

In [ ]:
df_sample = df[df.Mass < 20000]
plt.figure(figsize=(16, 12))
plt.scatter(df_sample.Mass, df_sample.RT)
x = df_sample.Mass.sort_values()
plt.plot(x, f_hi(x))
plt.plot(x, f_lo(x))
